# 版本记录

- v0.0.1  
初始版本

- v0.0.2  
原来的模型不够简洁，去掉匹配规则对象，全部换成标记符对象。  
考虑了一下，还是保留匹配规则对象，因为匹配规则，将来可能是重要的管理单元。  
再考虑了一下，准备通过运算符重载的方式增加匹配规则，暂时先不体现匹配规则对象。  
my parser有两种实现方案：
一种是利用操作符重载简洁的表达，形如：s += a| a&s   
另外一种采用函数方式表达可理解性更好，形如：s.add(a); s.add(a,s)  
这两种方式是等价的，我先用简洁的实现看看方不方便，如果不行再换成函数式  

初步实现了`S=a`的解析。

# 思路

- 左侧有且仅有一个非终结符  
    形如S=aS
- 匹配接口IMatch  
    实现了Match函数，Match函数负责识别一段文本，返回可能识别到的多个结果列表，如果没有识别到，返回(False,...)  
- 标记符对象  
    - 实现了IMatch接口  
    - 非终结符标记符对象  
        - 包括一个标记符对象列表，这些标记符对象之间是或的关系  
    - 终结符标记符对象  
        直接返回匹配结果就好，不包括匹配规则对象    
    - 开始符  
        特殊的非终结标记符，匹配就从它开始  

# 实现

## 引入及类型

In [31]:
from printobject import pp

In [32]:
from typing import List, Tuple

# 匹配之后，剩余的待匹配的结果类型描述，形如(True,[(2,3),(5,6)])，第一项表示是否匹配成功，第2项表示匹配的结果，可能有多个待匹配项
# 每一个匹配项第一项表示起始索引，第2项表示长度
ParseResults = Tuple[bool,List[Tuple[int,int]]]

## 匹配接口IMatch

In [33]:
class IMatch:
    # 表示待解析的原始字符串
    Text :str = None

    def Match(self, startIndex: int, length: int, isFullMatch: bool) -> ParseResults:
        '''
        功能描述：对待解析的字符串进行匹配，返回匹配的结果
        startIndex：待匹配字符串的起始索引
        length：待匹配字符串的长度
        isFullMatch：表示是否需要完全匹配，如果完全匹配，则必须字符串匹配完，才返回成功，只匹配部分，则匹配失败
        返回值：ParseResults类型，可能有多个剩余的待匹配项
        '''
        pass
    pass

## 标记符对象

In [34]:
class Symbol(IMatch):pass

### 终结符对象

In [35]:
class Terminal(Symbol):pass

### 非终结符对象

In [36]:
class NonTerminal(Symbol):
    def __init__(self):
        # 符号对象列表，这些符号对象是或的关系
        self.symbolList: List[Symbol] = []
    
    # 重载 `|=`操作符
    def __ior__(self,dst:Symbol):
        self.symbolList.append(dst)
        return self

### 关于结束符

开始似乎需要结束符，结束符表示最后空的匹配，才能匹配成功。后来思考一下，似乎不需要结束符了，因为对于每一个匹配规则对象，都要求完全匹配，不允许还剩余没有匹配的部分。所以，对于标记符对象、匹配规则对象，都要检查是否匹配完整。  

表示一个基础匹配单元，包括开始符、非终结标记符、终结标记符。  
```
# S是开始符，`aS`表示S的一个匹配规则，S、A都是非终结标记符，a是终结标记符
S=aS
A=aB
```

### 开始符对象

In [37]:
class Starter(NonTerminal):
    def __init__(self):
        super(Starter, self).__init__()
        # 匹配成功的次数
        self.successNum = 0
            
    def Match(self, startIndex: int, length: int, isFullMatch: bool) -> ParseResults:
        for symbol in self.symbolList:
            # 开始符对象的match必须全部匹配
            res = symbol.Match(startIndex, length, True)
            if res[0]==True: self.successNum += 1
                
    def Run(self, txt: str) -> int:
        '''
        txt：要匹配的文本
        返回匹配成功的次数，如果为0，表示没有匹配成功，如果大于1，表示有多个成功的匹配
        '''
        IMatch.Text = txt
        self.Match(0,len(IMatch.Text),True)
        return self.successNum

# 应用

## 实现`S=a`

## 字母`a`的终结符对象

In [38]:
class A(Terminal):
    def Match(self, startIndex: int, length: int, isFullMatch: bool) -> ParseResults:
        if length <= 0: return (False,None)
        
        # 如果严格匹配，只能匹配单字符
        if isFullMatch==True and length > 1: return (False,None)
        
        if IMatch.Text[0] != 'a': return (False,None)
        return (True,[(startIndex+1,length-1)])

# 试验运行

In [39]:
def Run():
    s=Starter()
    s |= A()
    res = s.Run('a')
    assert(res==1)

In [40]:
Run()